In [1]:
import numpy as np
path_list = ['1,2,4,6',
      '1,2,4,7',
      '1,3,5,8']

In [2]:
sequence_list = ['1,try,valid,lastname,click,commit,notobserve,errormessage',
'1,try,blank,lastname,click,commit,notobserve,errormessage',
'1,try,invalid_special_characters,lastname,click,commit,observe,errormessage',
'1,try,invalid_long,lastname,click,commit,observe,errormessage',
        '12,try,valid,pin1,click,notobserve,errormessage',
        '12,try,valid,pin2,click,notobserve,errormessage',
        '13,try,invalid,password,length,notobserve,errormessage',
        '13,try,valid,password,length,observe,errormessage',
        '135,auto,invalid,pin2,click,notobserve,errormessage',
        '135,auto,valid,pin2,click,notobserve,errormessage',
      '124,try,valid,extra_info1,click,observe,errormessage',
        '124,try,correct,extra_info2,select,notobserve,errormessage',
      '1358,pincode,correct,extra_info2,click,notobserve,errormessage',
       '1247,try,notcorrect,help,click,notobserve,errormessage',
       '1247,try,correct,help,click,notobserve,errormessage',
       '1246,auto,valid,dragdrop,click,observe,errormessage',
       '1246,auto,valid,dragdrop,click,notobserve,errormessage']

In [3]:
def data_dicts(X_):
    word_to_ix={}
    ix_to_word={}
    x=0
    for partial_quesries in X_:
        partial_quesries = partial_quesries.split(",")
        for word in partial_quesries:
            if word not in word_to_ix.keys():
                word_to_ix[word]=x
                ix_to_word[x]=word
                x+=1
    return (word_to_ix,ix_to_word)
location_to_state,state_to_location=data_dicts(path_list)    
location_to_state1,state_to_location1=data_dicts(sequence_list)

In [4]:
def list_to_num_seq(x, loc_to_state):
    token_seq = []
    for i in x:
        i = i.split(",")
        token_intermediate = []
        for j in i:
            token_intermediate.append(loc_to_state[j])
        token_seq.append(token_intermediate)
    return token_seq

In [5]:
path_list, sequence_list

(['1,2,4,6', '1,2,4,7', '1,3,5,8'],
 ['1,try,valid,lastname,click,commit,notobserve,errormessage',
  '1,try,blank,lastname,click,commit,notobserve,errormessage',
  '1,try,invalid_special_characters,lastname,click,commit,observe,errormessage',
  '1,try,invalid_long,lastname,click,commit,observe,errormessage',
  '12,try,valid,pin1,click,notobserve,errormessage',
  '12,try,valid,pin2,click,notobserve,errormessage',
  '13,try,invalid,password,length,notobserve,errormessage',
  '13,try,valid,password,length,observe,errormessage',
  '135,auto,invalid,pin2,click,notobserve,errormessage',
  '135,auto,valid,pin2,click,notobserve,errormessage',
  '124,try,valid,extra_info1,click,observe,errormessage',
  '124,try,correct,extra_info2,select,notobserve,errormessage',
  '1358,pincode,correct,extra_info2,click,notobserve,errormessage',
  '1247,try,notcorrect,help,click,notobserve,errormessage',
  '1247,try,correct,help,click,notobserve,errormessage',
  '1246,auto,valid,dragdrop,click,observe,errormes

In [6]:
path_array = np.array(list_to_num_seq(path_list, location_to_state)) # for lst
sequence_array = np.array(list_to_num_seq(sequence_list, location_to_state1)) # for lst2

In [7]:
sequence_array

array([list([0, 1, 2, 3, 4, 5, 6, 7]), list([0, 1, 8, 3, 4, 5, 6, 7]),
       list([0, 1, 9, 3, 4, 5, 10, 7]), list([0, 1, 11, 3, 4, 5, 10, 7]),
       list([12, 1, 2, 13, 4, 6, 7]), list([12, 1, 2, 14, 4, 6, 7]),
       list([15, 1, 16, 17, 18, 6, 7]), list([15, 1, 2, 17, 18, 10, 7]),
       list([19, 20, 16, 14, 4, 6, 7]), list([19, 20, 2, 14, 4, 6, 7]),
       list([21, 1, 2, 22, 4, 10, 7]), list([21, 1, 23, 24, 25, 6, 7]),
       list([26, 27, 23, 24, 4, 6, 7]), list([28, 1, 29, 30, 4, 6, 7]),
       list([28, 1, 23, 30, 4, 6, 7]), list([31, 20, 2, 32, 4, 10, 7]),
       list([31, 20, 2, 32, 4, 6, 7])], dtype=object)

In [8]:
gamma = 0.75 # Discount factor 
alpha = 0.9 # Learning rate

In [9]:
def gen_reward_matrix(a_array,max_elem):
    t= np.zeros((max_elem+3,max_elem+3))
    di = dict()
    for i in range(0,len(a_array)-1):
        if a_array[i] in di.keys():
            di[a_array[i]].append(a_array[i+1])
        else:
            di[a_array[i]] = a_array[i+1]
    if a_array[-1] not in di:
        di[a_array[-1]]=a_array[-1]+1
    else:
        di[a_array[-1]].append(a_array[-2])
    
    for i in range(max_elem+3):
        if i in di.keys():
            ls = di[i] 
            t[i][di[i]]=1
    return t

In [10]:
def obtain_sequence_pattern(a_arr,start_location,end_location,location_to_state):
    for i in a_arr:
        if (location_to_state[start_location] in i) and (location_to_state[end_location] in i):
            a = i
            break
    return a
    

In [11]:
import random
def get_optimal_route(start_location,end_location,location_to_state,state_to_location, a_arr):
    a_arr = sorted(a_arr, key=lambda k: random.random())
    a_array = obtain_sequence_pattern(a_arr,start_location,end_location,location_to_state)
    max_elem = max(a_array)
    rewards = gen_reward_matrix(a_array,max(a_array))
    rewards_new = np.copy(rewards)
    ending_state = location_to_state[end_location]
    rewards_new[ending_state,ending_state] = 999
    Q = np.array(np.zeros([max_elem+1,max_elem+1]))

    for i in range(1000):
        current_state = np.random.randint(0,max_elem+1) # Python excludes the upper bound
        playable_actions = []
        for j in range(max_elem+1):
            if rewards_new[current_state,j] > 0:
                playable_actions.append(j)
        if playable_actions:
            next_state = np.random.choice(playable_actions)
            TD = rewards_new[current_state,next_state] + gamma * Q[next_state, np.argmax(Q[next_state,])] - Q[current_state,next_state]
            Q[current_state,next_state] += alpha * TD
  
    route = [start_location]
    
    next_location = start_location
    
    while(next_location != end_location):
        starting_state = location_to_state[start_location]    
        
        next_state = np.argmax(Q[starting_state,])
    
        next_location = state_to_location[next_state]
        route.append(next_location)
        start_location = next_location
    return route

In [12]:
get_optimal_route('1','7',location_to_state,state_to_location,path_array)

['1', '2', '4', '7']

In [13]:
get_optimal_route('1','errormessage',location_to_state1,state_to_location1,sequence_array)

['1',
 'try',
 'invalid_long',
 'lastname',
 'click',
 'commit',
 'observe',
 'errormessage']

In [14]:
result_lst = []
for i in range(10):
    result_lst.append(get_optimal_route('1','errormessage',location_to_state1,state_to_location1,sequence_array))

seq1 = [list(item) for item in set(tuple(row) for row in result_lst)]

In [15]:
seq1

[['1',
  'try',
  'invalid_long',
  'lastname',
  'click',
  'commit',
  'observe',
  'errormessage'],
 ['1',
  'try',
  'valid',
  'lastname',
  'click',
  'commit',
  'notobserve',
  'errormessage'],
 ['1',
  'try',
  'invalid_special_characters',
  'lastname',
  'click',
  'commit',
  'observe',
  'errormessage'],
 ['1',
  'try',
  'blank',
  'lastname',
  'click',
  'commit',
  'notobserve',
  'errormessage']]

In [16]:
for i in seq1:
    i[0]=get_optimal_route(i[0][0],i[0][-1],location_to_state,state_to_location,path_array)

In [17]:
seq1

[[['1'],
  'try',
  'invalid_long',
  'lastname',
  'click',
  'commit',
  'observe',
  'errormessage'],
 [['1'],
  'try',
  'valid',
  'lastname',
  'click',
  'commit',
  'notobserve',
  'errormessage'],
 [['1'],
  'try',
  'invalid_special_characters',
  'lastname',
  'click',
  'commit',
  'observe',
  'errormessage'],
 [['1'],
  'try',
  'blank',
  'lastname',
  'click',
  'commit',
  'notobserve',
  'errormessage']]

In [22]:
result_lst = []
for i in range(30):
    result_lst.append(get_optimal_route('try','errormessage',location_to_state1,state_to_location1,sequence_array))

seq_try = [list(item) for item in set(tuple(row) for row in result_lst)]

In [23]:
sequence_list

['1,try,valid,lastname,click,commit,notobserve,errormessage',
 '1,try,blank,lastname,click,commit,notobserve,errormessage',
 '1,try,invalid_special_characters,lastname,click,commit,observe,errormessage',
 '1,try,invalid_long,lastname,click,commit,observe,errormessage',
 '12,try,valid,pin1,click,notobserve,errormessage',
 '12,try,valid,pin2,click,notobserve,errormessage',
 '13,try,invalid,password,length,notobserve,errormessage',
 '13,try,valid,password,length,observe,errormessage',
 '135,auto,invalid,pin2,click,notobserve,errormessage',
 '135,auto,valid,pin2,click,notobserve,errormessage',
 '124,try,valid,extra_info1,click,observe,errormessage',
 '124,try,correct,extra_info2,select,notobserve,errormessage',
 '1358,pincode,correct,extra_info2,click,notobserve,errormessage',
 '1247,try,notcorrect,help,click,notobserve,errormessage',
 '1247,try,correct,help,click,notobserve,errormessage',
 '1246,auto,valid,dragdrop,click,observe,errormessage',
 '1246,auto,valid,dragdrop,click,notobserve,e

In [24]:
seq_try

[['try', 'notcorrect', 'help', 'click', 'notobserve', 'errormessage'],
 ['try', 'correct', 'help', 'click', 'notobserve', 'errormessage'],
 ['try', 'valid', 'lastname', 'click', 'commit', 'notobserve', 'errormessage'],
 ['try', 'valid', 'password', 'length', 'observe', 'errormessage'],
 ['try', 'correct', 'extra_info2', 'select', 'notobserve', 'errormessage'],
 ['try', 'blank', 'lastname', 'click', 'commit', 'notobserve', 'errormessage'],
 ['try',
  'invalid_special_characters',
  'lastname',
  'click',
  'commit',
  'observe',
  'errormessage'],
 ['try',
  'invalid_long',
  'lastname',
  'click',
  'commit',
  'observe',
  'errormessage'],
 ['try', 'invalid', 'password', 'length', 'notobserve', 'errormessage'],
 ['try', 'valid', 'pin1', 'click', 'notobserve', 'errormessage']]

In [25]:
result_lst = []
for i in range(100):
    result_lst.append(get_optimal_route('1358','errormessage',location_to_state1,state_to_location1,sequence_array))

seq2 = [list(item) for item in set(tuple(row) for row in result_lst)]

for i in seq2:
    i[0]=get_optimal_route(i[0][0],i[0][-1],location_to_state,state_to_location,path_array)

In [26]:
seq2

[[['1', '3', '5', '8'],
  'pincode',
  'correct',
  'extra_info2',
  'click',
  'notobserve',
  'errormessage']]

In [27]:
result_lst = []
for i in range(100):
    result_lst.append(get_optimal_route('135','errormessage',location_to_state1,state_to_location1,sequence_array))

seq3 = [list(item) for item in set(tuple(row) for row in result_lst)]

for i in seq3:
    i[0]=get_optimal_route(i[0][0],i[0][-1],location_to_state,state_to_location,path_array)

In [28]:
seq3

[[['1', '3', '5'],
  'auto',
  'valid',
  'pin2',
  'click',
  'notobserve',
  'errormessage'],
 [['1', '3', '5'],
  'auto',
  'invalid',
  'pin2',
  'click',
  'notobserve',
  'errormessage']]

In [29]:
def get_unique_seq(alist):
    x = []
    for i in alist:
        i = i.split(",")
        x.append([i[0],i[-1]])
    x = [list(item) for item in set(tuple(row) for row in x)]
    return x

In [30]:
def obtain_all_sequences():    
    uniq = get_unique_seq(sequence_list)
    seq_i = []
    for i in uniq:
        print(i)
        result_lst = []
        for j in range(20):
            result_lst.append(get_optimal_route(i[0],i[-1],location_to_state1,state_to_location1,sequence_array))
        only_unique = [list(item) for item in set(tuple(row) for row in result_lst)]
        seq_i.append(only_unique)
    return seq_i

In [31]:
seq_i = obtain_all_sequences()

['12', 'errormessage']
['1246', 'errormessage']
['1247', 'errormessage']
['135', 'errormessage']
['1', 'errormessage']
['13', 'errormessage']
['1358', 'errormessage']
['124', 'errormessage']


In [32]:
seq_i

[[['12', 'try', 'valid', 'pin1', 'click', 'notobserve', 'errormessage'],
  ['12', 'try', 'valid', 'pin2', 'click', 'notobserve', 'errormessage']],
 [['1246', 'auto', 'valid', 'dragdrop', 'click', 'notobserve', 'errormessage'],
  ['1246', 'auto', 'valid', 'dragdrop', 'click', 'observe', 'errormessage']],
 [['1247', 'try', 'correct', 'help', 'click', 'notobserve', 'errormessage'],
  ['1247',
   'try',
   'notcorrect',
   'help',
   'click',
   'notobserve',
   'errormessage']],
 [['135', 'auto', 'valid', 'pin2', 'click', 'notobserve', 'errormessage'],
  ['135', 'auto', 'invalid', 'pin2', 'click', 'notobserve', 'errormessage']],
 [['1',
   'try',
   'invalid_long',
   'lastname',
   'click',
   'commit',
   'observe',
   'errormessage'],
  ['1',
   'try',
   'valid',
   'lastname',
   'click',
   'commit',
   'notobserve',
   'errormessage'],
  ['1',
   'try',
   'invalid_special_characters',
   'lastname',
   'click',
   'commit',
   'observe',
   'errormessage'],
  ['1',
   'try',
   'b

In [33]:
def build_dict():
    dictionary = dict()
    uniq = get_unique_seq(sequence_list)
    for i in uniq:
        i = int(i[0])
        dictionary[i]={"observe":[],"notobserve":[]}
    return dictionary
final_res_dict = build_dict()
final_res_dict

{12: {'observe': [], 'notobserve': []},
 1246: {'observe': [], 'notobserve': []},
 1247: {'observe': [], 'notobserve': []},
 135: {'observe': [], 'notobserve': []},
 1: {'observe': [], 'notobserve': []},
 13: {'observe': [], 'notobserve': []},
 1358: {'observe': [], 'notobserve': []},
 124: {'observe': [], 'notobserve': []}}

In [34]:
def obtain_result(dictionary):
    for i in range(len(seq_i)):
        for j in range(len(seq_i[i])):
            if "notobserve" in seq_i[i][j]:
                dictionary[int(seq_i[i][j][0])]["notobserve"].append(seq_i[i][j][1:])
            elif "observe" in seq_i[i][j]:
                dictionary[int(seq_i[i][j][0])]["observe"].append(seq_i[i][j][1:])
    return dictionary
        

In [35]:
final_res_dict = obtain_result(final_res_dict)
final_res_dict

{12: {'observe': [],
  'notobserve': [['try',
    'valid',
    'pin1',
    'click',
    'notobserve',
    'errormessage'],
   ['try', 'valid', 'pin2', 'click', 'notobserve', 'errormessage']]},
 1246: {'observe': [['auto',
    'valid',
    'dragdrop',
    'click',
    'observe',
    'errormessage']],
  'notobserve': [['auto',
    'valid',
    'dragdrop',
    'click',
    'notobserve',
    'errormessage']]},
 1247: {'observe': [],
  'notobserve': [['try',
    'correct',
    'help',
    'click',
    'notobserve',
    'errormessage'],
   ['try', 'notcorrect', 'help', 'click', 'notobserve', 'errormessage']]},
 135: {'observe': [],
  'notobserve': [['auto',
    'valid',
    'pin2',
    'click',
    'notobserve',
    'errormessage'],
   ['auto', 'invalid', 'pin2', 'click', 'notobserve', 'errormessage']]},
 1: {'observe': [['try',
    'invalid_long',
    'lastname',
    'click',
    'commit',
    'observe',
    'errormessage'],
   ['try',
    'invalid_special_characters',
    'lastname',
    '

In [37]:
import pickle

with open('result.pickle','wb') as handle:
    pickle.dump(final_res_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)